In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
pd.set_option('display.max_columns', None) # 显示所有列
pd.set_option('display.max_rows', 10) # 设定df全部显示的行数

np.random.seed(12345)
np.set_printoptions(precision=4, suppress=True)

# 一个cell做多个输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

## 1 pandas与模型代码的接口 
> 最好当数据是均匀的时候使用.values属性。例如，全是数值类型。如果数据是不均匀的，结果会是Python对象的ndarray;其实就是某一列的数据类型与其他的不一致

pandas与其它分析库通常是靠NumPy的数组联系起来的

In [6]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
              'x1': [0.01, -0.01, 0.25, -4.1, 0.],
              'y': [-1.5, 0., 3.6, 1.3, -2.]})

data
data.columns
data.values

df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])
df2

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


Index(['x0', 'x1', 'y'], dtype='object')

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [7]:
df3 = data.copy()
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3
df3.values

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [8]:
# 数据选取
model_cols = ['x0', 'x1']
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [10]:
# 类别数据选取
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'], categories=['a', 'b'])
data

# 替换category列为虚变量，我们可以创建虚变量，删除category列，然后添加到结果
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## 2 用Patsy创建模型描述（了解即可）
该库比较少用，更倾向于数学表达式的调用，需另外从不同的角度补充；
- patsy.dmatrices函数接收一个公式字符串和一个数据集（可以是DataFrame或数组的字典），为线性模型创建设计矩阵
- patsy.build_design_matrices函数可以使用原始样本数据集的保存信息，来转换新数据

In [14]:
import patsy

data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
              'x1': [0.01, -0.01, 0.25, -4.1, 0.],
              'y': [-1.5, 0., 3.6, 1.3, -2.]})

data

y, X = patsy.dmatrices('y ~ x0 + x1', data)
y
X

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [16]:
np.asarray(y)
np.asarray(X)

patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

### 2.1 用Patsy公式进行数据转换
> 可以将Python代码与patsy公式结合。在评估公式时，库将尝试查找在封闭作用域内使用的函数

- 1、常见的变量转换包括标准化（平均值为0，方差为1）和中心化（减去平均值）。Patsy有内置的函数进行这样的工作；
- 2、patsy.build_design_matrices函数可以使用原始样本数据集的保存信息，来转换新数据
- 3、Patsy中的加号不是加法的意义，当按照名称将数据集的列相加时，必须用特殊I函数将它们封装

In [19]:
# 1、简单的数据转换
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

# 2、patsy 的数据标准化
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

# 3、patsy.build_design_matrices函数进行不同数据集的转换
new_data = pd.DataFrame({'x0': [6, 7, 8, 9],
                'x1': [3.1, -0.5, 0, 2.3],
                'y': [1, 2, 3, 4]})

new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

# 4、实现+本身的含义
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

### 2.2 分类数据和Patsy
> 该部分书中介绍的很少，再加上Patsy库不是必须掌握，该部分了解即可，不再另外补充

在Patsy公式中使用非数值数据，它们会默认转换为虚变量

In [21]:
data = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
              'key2': [0, 1, 0, 1, 0, 1, 0, 0],
              'v1': [1, 2, 3, 4, 5, 6, 7, 8],
              'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]})

y, X = patsy.dmatrices('v2 ~ key1', data)
X

# 忽略模型截距
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

# ANVOA的方差分析
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


## 3 statsmodels介绍
> 该模块非常重要，有时间需系统学习该模块并在有道云笔记中做好详细记录

Statsmodels包含许多经典的统计方法，但没有贝叶斯方法和机器学习模型。

**包含的模型：**
- 线性模型、广义线性模型和健壮线性模型
- 线性混合效应模型
- **方差（ANOVA）方法分析**
- **时间序列过程和状态空间模型**
- 广义矩估计

### 3.1 估计线性模型
statsmodels的线性模型有两种不同的接口：基于内置API和基于公式（结合patsy）；


In [23]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 生成线性模型的随机数据
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
        dnorm(0, 0.6, size=N),
        dnorm(0, 0.2, size=N)]

eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]  # 模式的可知真实参数

y = np.dot(X, beta) + eps

#### 3.1.1 基于内置函数的模型搭建和评估
该部分主要使用的statsmodels的内置函数

In [24]:
# sm.add_constant函数可以添加一个截距的列到现存的矩阵
X_model = sm.add_constant(X)
X_model[:5]

# 拟合普通的最小二乘线性回归
model = sm.OLS(y, X)
results = model.fit() # 拟合该模型
results.params

# 对结果使用summary方法可以打印模型的详细诊断结果
print(results.summary())

array([[ 1.    ,  0.5955, -0.5901,  0.3391],
       [ 1.    ,  0.3373, -0.2569, -0.2306],
       [ 1.    ,  0.2253, -1.3566, -0.2644],
       [ 1.    , -0.0064,  0.4871,  0.401 ],
       [ 1.    ,  0.8954,  0.2188, -0.4344]])

array([0.1236, 0.2434, 0.4947])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     24.91
Date:                Mon, 11 Mar 2019   Prob (F-statistic):           4.93e-12
Time:                        13:52:23   Log-Likelihood:                -22.935
No. Observations:                 100   AIC:                             51.87
Df Residuals:                      97   BIC:                             59.68
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1236      0.049      2.507      0.0

####  3.1.2 基于公式API和Patsy的公式字符串

In [29]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params
results.tvalues

,col0,col1,col2,y
0,0.595502,-0.590117,0.339078,0.057067
1,0.337329,-0.256869,-0.230576,0.185299
2,0.225323,-1.356563,-0.264394,-0.716250
3,-0.006397,0.487139,0.401037,0.338845
4,0.895401,0.218825,-0.434440,0.154862


Intercept    0.023931
col0         0.123297
col1         0.244316
col2         0.497207
dtype: float64

Intercept    0.771760
col0         2.494960
col1         5.495030
col2         6.905723
dtype: float64

### 3.2 估计时间序列过程
> 该部分非常重要，详细文档请参看有道云笔记中时间序列预测部分

statsmodels的另一模型类是进行时间序列分析，包括自回归过程、卡尔曼滤波和其它态空间模型，和多元自回归模型；（需提前做各种验证，数据分布必须满足该系列模型特征）

In [30]:
# 用自回归结构和噪声来模拟一些时间序列数据
init_x = 4

import random
values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [31]:
# 使用自回归拟合该模型
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)

results.params

array([ 0.    ,  0.7906, -0.4096, -0.0198,  0.0103,  0.0107])

## 4 scikit-learn介绍
> sklearn是非常重要的ML集成库，每一个小模块都可以单独详细划分出来整理，且该库需要不断的使用和总结；相关笔记见有道云的sklearn主题系列

In [33]:
train = pd.read_csv('./datasets/titanic/train.csv')
test = pd.read_csv('./datasets/titanic/test.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
# 缺失值查询
train.isnull().sum()
test.isnull().sum()

# 缺失值填充
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

# 哑变量编码
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

# 特征值选取
predictors = ['Pclass', 'IsFemale', 'Age']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values
X_train[:5]
y_train[:5]

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
              ... 
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Length: 12, dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
              ... 
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Length: 11, dtype: int64

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

array([0, 1, 1, 1, 0], dtype=int64)

In [36]:
# 使用默认参数创建模型
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

# 使用训练数据 拟合模型
model.fit(X_train, y_train)

# 使用测试数据 验证模型
y_predict = model.predict(X_test)
y_predict[:10]

# 准确率计算（sklearn有相关的子模块）
# (y_true == y_predict).mean()

D:\Anaconda2_5.1.0_setup\envs\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

> 交叉验证通过分割训练数据来模拟样本外预测。基于模型的精度得分（比如均方差），
可以对模型参数进行网格搜索。有些模型，如logistic回归，有内置的交叉验证的估计
类。例如，logisticregressioncv类可以用一个参数指定网格搜索对模型的正则化参数C
的粒度：

交叉验证寻找最优参数主要有两个方向：内置和通用，首先确定是否有内置的交叉验证估计

In [37]:
# 内置的交叉验证（部分模型存在）
from sklearn.linear_model import LogisticRegressionCV
model_cv = LogisticRegressionCV(10)
model_cv.fit(X_train, y_train)

D:\Anaconda2_5.1.0_setup\envs\python36\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [38]:
# 通用的交叉验证模型
from sklearn.model_selection import cross_val_score
model = LogisticRegression(C=10)
scores = cross_val_score(model, X_train, y_train, cv=4)
scores

D:\Anaconda2_5.1.0_setup\envs\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda2_5.1.0_setup\envs\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda2_5.1.0_setup\envs\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda2_5.1.0_setup\envs\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.7723, 0.8027, 0.7703, 0.7883])